In [1]:
pip install gradio pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gradio as gr

In [5]:
# Load the dataset
file_path = "/content/Diseases_Foods_Dataset_Real.csv"
df = pd.read_csv(file_path)
data = df

In [6]:
# Function to dynamically ask questions based on diseases
def ask_followup_questions(disease_input):
    diseases = [d.strip().lower() for d in disease_input.split(",")]
    followup_questions = {}

    if "diabetes" in diseases:
        followup_questions["Sugar Level (mg/dL)"] = "Enter your sugar level (or type 'Don't Know')"
    if "hypertension" in diseases:
        followup_questions["Blood Pressure Level"] = "Enter your blood pressure level (or type 'Don't Know')"
    if "kidney failure" in diseases:
        followup_questions["Water Intake"] = "Enter your daily water intake (liters) (or type 'Don't Know')"

    return followup_questions

In [7]:
# Function to check food recommendations
def check_food_recommendations(disease_input, food_input, sugar_level=None, bp_level=None, water_intake=None):
    diseases = [d.strip().lower() for d in disease_input.split(",")]
    foods = [f.strip().lower() for f in food_input.split(",")]
    results = ""

    for disease in diseases:
        disease_data = df[df["Disease"].str.lower() == disease]

        if disease_data.empty:
            results += f"\n⚠️ No data found for disease: {disease}.\n"
            continue

        not_recommended = disease_data.iloc[0]["Foods/Ingredients Not Recommended"].lower().split(", ")
        recommended = disease_data.iloc[0]["Recommended Foods"].lower().split(", ")
        total_calories = disease_data.iloc[0]["Total Calories of Not Recommended Foods"]

        results += f"\n Food Recommendations for {disease.title()} \n"

        for food in foods:
            if food in not_recommended:
                results += f" {food} is NOT recommended for {disease}.\n"
                results += f"⚠️ Total Calories of Not Recommended Foods: {total_calories} kcal\n"
            else:
                results += f"✅ {food} is safe to consume for {disease}.\n"

        results += f"\n🍏 Recommended Foods: {', '.join(recommended)}\n"
        results += "-" * 50 + "\n"

    return results


In [8]:
# Create Gradio UI
with gr.Blocks() as ui:
    gr.Markdown("## 🍏 AI-Powered Food Recommendation System")

    disease_input = gr.Textbox(label="Enter Diseases (comma-separated)")
    food_input = gr.Textbox(label="Enter Foods (comma-separated)")

    sugar_level_input = gr.Textbox(label="Sugar Level (if Diabetes)", visible=False)
    bp_level_input = gr.Textbox(label="Blood Pressure (if Hypertension)", visible=False)
    water_intake_input = gr.Textbox(label="Water Intake (if Kidney Failure)", visible=False)

    submit_btn = gr.Button("🔍 Check Food Safety")
    output = gr.Textbox(label="Recommendation")

    # Function to update visibility of inputs
    disease_input.change(ask_followup_questions, inputs=disease_input, outputs=[sugar_level_input, bp_level_input, water_intake_input])

    submit_btn.click(
        check_food_recommendations,
        inputs=[disease_input, food_input, sugar_level_input, bp_level_input, water_intake_input],
        outputs=output
    )

ui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eab39fe68eee88edc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
